# Analysis of the Radiometrics Features

In [19]:
import pandas as pd
import numpy as np

In [20]:
# parameteres
binwidth = 20
sigma = [1,2,3]

sm_radiometrics_file = f'santamaria_data_all__binwidth_{binwidth}_sigma_{sigma}.csv' 
sm_radiometrics = pd.read_csv(sm_radiometrics_file)

In [21]:
sm_radiometrics.head()

,PATIENT_ID,SEXO_MASCULINO,EDAD,FECHA_CIRUGIA,BIOPSIA_QX_PULMONAR,BIOPSIA_FBC-EBUS,BIOPSIA_OTRO_SITIO,RESULTADO_BP,BP_COMPLETA,HISTOLOGIA,...,torax3d_original_shape_Maximum2DDiameterColumn,torax3d_original_shape_Maximum2DDiameterRow,torax3d_original_shape_Maximum2DDiameterSlice,torax3d_original_shape_Maximum3DDiameter,torax3d_original_shape_MeshVolume,torax3d_original_shape_MinorAxisLength,torax3d_original_shape_Sphericity,torax3d_original_shape_SurfaceArea,torax3d_original_shape_SurfaceVolumeRatio,torax3d_original_shape_VoxelVolume
0,sm_001,1,70,NaN,NO,SI,NO,BIOPSIAS TRANSBRONQUIALES-ENDOBRONQUIALES: VAR...,Fecha Informe : 26/03/2018\n\n \n\nINF...,ADENOCARCINOMA,...,77.951427,71.542106,74.982329,84.533770,55279.552157,49.332435,0.340003,20640.080294,0.373376,55429.621807
1,sm_002,0,49,2017-05-19 00:00:00,SI,SI,NO,1.- PUNCION ENDOSONOGRAFICA DE LINFONODOS 4L: ...,Fecha Informe : 17/02/2017\n \n\n\n\nI...,ADENOCARCINOMA,...,32.497872,30.012252,33.959077,34.225753,8613.947922,24.832407,0.637823,3186.045702,0.369871,8634.885267
2,sm_003,0,43,2017-07-05 00:00:00,NO,NO,SI,1.- VENTANA AORTOPULMUNAR : METASTASIS DE ADEN...,Fecha Informe : 10/07/2017\n\n\n\nINFORME ANAT...,ADENOCARCINOMA,...,15.506804,14.499420,18.110513,18.164537,956.215685,10.689812,0.774616,605.947428,0.633693,965.217617
3,sm_004,0,54,2017-05-05 00:00:00,SI,SI,NO,2.- PUNCION EBUS LINFONODO 4L: ABUNDANTE MATER...,Fecha Informe : 04/01/2017\n\nINFORME ANATOMO-...,ADENOCARCINOMA,...,52.867093,38.342800,51.380028,56.218770,21895.758462,28.599848,0.659739,5737.009500,0.262015,21915.678306
4,sm_005,0,44,2016-01-06 00:00:00,SI,NO,SI,2.- LOBECTOMIA SUPERIOR IZQUIERDA (124 GR.): A...,Fecha Informe : 11 DE ENERO DE 2016\n\nINFORME...,ADENOCARCINOMA,...,45.658721,47.321523,45.846671,48.005364,26342.882775,33.996793,0.552775,7745.406975,0.294023,26405.130223


In [11]:
sm_radiometrics.drop(columns=['FECHA_CIRUGIA', 'RESULTADO_BP', 'BP_COMPLETA', 'HISTOLOGIA', 'MUTACION_EGFR', 'COMENTARIO', 'PET_SEG', 'BODY_CT_SEG'])

Index(['PATIENT_ID', 'SEXO_MASCULINO', 'EDAD', 'FECHA_CIRUGIA',
       'BIOPSIA_QX_PULMONAR', 'BIOPSIA_FBC-EBUS', 'BIOPSIA_OTRO_SITIO',
       'RESULTADO_BP', 'BP_COMPLETA', 'HISTOLOGIA', 'EGFR', 'MUTACION_EGFR',
       'ALK', 'MUTACION_ALK', 'PDL-1', 'MUTACION_PDL-1', 'ROS', 'MUTACION_ROS',
       'ADENOPATIAS', 'STAGE', 'RECIDIVA', 'COMENTARIO', 'SUV', 'TAMAÑO_BP_mm',
       'TAMAÑO_CT_mm', '3D_TORAX_SEG', 'PET_SEG', 'BODY_CT_SEG', 'IV CONTRAST',
       'body_diagnostics_Configuration_EnabledImageTypes',
       'body_diagnostics_Configuration_Settings',
       'body_diagnostics_Image-original_Dimensionality',
       'body_diagnostics_Image-original_Hash'],
      dtype='object')